# 1、线性回归实现

In [2]:
import random
import torch
from torch.utils import data
from d2l import torch as d2l

## 1.1 生成数据集

In [3]:
true_w = torch.tensor([2, -3.4])  #真是的权重
true_b = 4.2  #真实的偏置
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
features

tensor([[-0.8552, -0.1741],
        [-0.1493, -1.9095],
        [ 1.6861,  0.9948],
        ...,
        [-1.4740,  0.0132],
        [ 1.6042, -0.4986],
        [-0.4885,  0.0733]])

## 1.2 读取数据集

In [4]:
def load_array(data_arrays,batch_size,is_train=True):
    '''构造一个torch的数据迭代器'''
    dataset=data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)  #是否打乱顺序

batch_size=10
data_iter=load_array((features,labels),batch_size)
next(iter(data_iter))

[tensor([[ 0.2609, -0.9834],
         [ 0.7405, -1.1951],
         [-1.0310, -0.6925],
         [-0.9030, -0.6420],
         [-0.1177,  1.0876],
         [ 0.3032, -0.4538],
         [ 0.3410,  0.0370],
         [ 0.4416, -1.1808],
         [-0.2002,  0.2278],
         [-0.5243, -0.5270]]),
 tensor([[8.0464],
         [9.7477],
         [4.5073],
         [4.5987],
         [0.2512],
         [6.3382],
         [4.7636],
         [9.1139],
         [3.0103],
         [4.9568]])]

## 1.3 定义模型

In [5]:
# 第一个时输入的特征形状2，第二个是输出特征形状1
net=torch.nn.Sequential(torch.nn.Linear(2,1))

## 1.4 初始化模型参数
    在使用net之前，我们需要初始化模型参数。 如在线性回归模型中的权重和偏置。 深度学习框架通常有预定义的方法来初始化参数。 在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零。

正如我们在构造nn.Linear时指定输入和输出尺寸一样， 现在我们能直接访问参数以设定它们的初始值。 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。

In [6]:
net[0].weight.data.normal_(0,0.01)  #权重
net[0].bias.data.fill_(0)  #偏置

tensor([0.])

## 1.5 定义损失误差

In [7]:
#默认情况下，返回所有样本的损失的平均值（均方误差）
loss=torch.nn.MSELoss()
#loss=torch.nn.MSELoss(reduction='sum')  #以损失值的和作为损失函数

## 1.6 定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。 当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

In [8]:
trainer=torch.optim.SGD(net.parameters(), lr=0.02)

## 1.7 训练
回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（train_data）， 不停地从中获取一个小批量的输入和相应的标签。 对于每一个小批量，我们会进行以下步骤:
*   通过调用net(X)生成预测并计算损失l（前向传播）。
*   通过进行反向传播来计算梯度。
*   通过调用优化器来更新模型参数。

In [9]:
num_epoch=3
for epoch in range(num_epoch):
    for x,y in data_iter:  
        l=loss(net(x),y)  #计算每次训练的损失
        trainer.zero_grad()  #清楚梯度内存
        l.backward()  #方向传播计算梯度
        trainer.step() #trainer步进 根据指定的优化算法进行参数的寻优
    l=loss(net(features),labels) #根据上面的参数优化结果计算参数对整个数据集的拟合状态 以loss进行反映
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.009539
epoch 2, loss 0.000112
epoch 3, loss 0.000108


下面我们比较生成数据集的真实参数和通过有限数据训练获得的模型参数。 要访问参数，我们首先从net访问所需的层，然后读取该层的权重和偏置。 正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。

In [10]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0003,  0.0004])
b的估计误差： tensor([-0.0007])


## 小结
如果将小批量的总损失替换为小批量损失的平均值，你需要如何更改学习率？